In [1]:
# !pip install tensorflow[and-cuda] tensorflow_hub tqdm opencv-python

In [3]:
# Set tensorflow to CPU
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import cv2
import tensorflow_hub as hub
import tensorflow as tf
from tqdm import tqdm
from pathlib import Path
from math import ceil


# Load model from hub
model = hub.load("https://www.kaggle.com/models/kaggle/esrgan-tf2/TensorFlow2/esrgan-tf2/1")

# DATASET_PATH = Path(r'/home/phdomingues/masters/data/UNIFESP/all')
DATASET_PATH = Path(r'/home/phdomingues/masters/data/UNIFESP/syntetic/synthetics')
OUTPUT_PATH = DATASET_PATH.parent / 'super_resolution'
OUTPUT_PATH.mkdir(exist_ok=True, parents=True)

BATCH_SIZE = 4
SAMPLE_SIZE = 5 # Numero de amostras que serao plotadas

IMG_SUFFIXES = ['.png', '.jpg', '.jpeg', '.bmp']

NameError: name 'hub' is not defined

In [2]:
def batch_array(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

## Image Super Resolution ([ISR library](https://github.com/idealo/image-super-resolution))

In [4]:
import numpy as np
from PIL import Image

img = Image.open('/mnt/d/ComputerScience/Mestrado/data/UNIFESP/syntetic/synthetics/00202.png')
lr_img = np.array(img)

from ISR.models import RDN

rdn = RDN(weights='psnr-small')
sr_img = rdn.predict(lr_img)
Image.fromarray(sr_img).save('/mnt/d/ComputerScience/Mestrado/data/UNIFESP/00202_super.png')

---

## Kaggle model (esrgan-tf2)

In [12]:
img_paths = [path.resolve() for path in DATASET_PATH.glob('*') if path.suffix in IMG_SUFFIXES]

for batch in tqdm(batch_array(img_paths, n=BATCH_SIZE), total=ceil(len(img_paths)/BATCH_SIZE)):
    imgs = [cv2.cvtColor(cv2.imread(str(img_path)), cv2.COLOR_BGR2RGB) for img_path in batch]
    
    # To add an extra dimension for batch, use tf.expand_dims()
    #low_resolution_image = tf.expand_dims(image) # Low Resolution Image of shape [batch_size, height, width, 3]
    low_resolution_image = tf.cast(imgs, tf.float32)
    super_resolution = model(low_resolution_image) # Perform Super Resolution here
    for img_path, img in tqdm(zip(batch, super_resolution), total=len(batch)):
        # Cast it back to uint8 before saving
        img = tf.cast(tf.clip_by_value(img, 0, 255), tf.uint8)
        cv2.imwrite(str(OUTPUT_PATH / img_path.name), cv2.cvtColor(img.numpy(), cv2.COLOR_RGB2BGR))

100%|██████████| 90/90 [20:11<00:00, 13.46s/it]
